In [1]:
import numpy as np # linear algebra

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error
data = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv' , nrows = 10000000)
# data["pickup_longitude"].describe()
# Remove data with extreme outlier coordinates or negative fares
data = data.query('pickup_latitude > 40.7 and pickup_latitude < 40.8 and ' +
                  'dropoff_latitude > 40.7 and dropoff_latitude < 40.8 and ' +
                  'pickup_longitude > -74 and pickup_longitude < -73.9 and ' +
                  'dropoff_longitude > -74 and dropoff_longitude < -73.9 and ' +
                  'fare_amount > 0'
                  )

y = data.fare_amount



# train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
# first_model = RandomForestRegressor(n_estimators=50, random_state=1).fit(train_X, train_y)

In [3]:
# create new features
data['abs_lon_change'] = abs(data.dropoff_longitude - data.pickup_longitude)
data['abs_lat_change'] = abs(data.dropoff_latitude - data.pickup_latitude)

features_2  = ['pickup_longitude',
               'pickup_latitude',
               'dropoff_longitude',
               'dropoff_latitude',
               'abs_lat_change',
               'abs_lon_change']

X = data[features_2]
new_train_X, new_val_X, new_train_y, new_val_y = train_test_split(X, y, random_state=1)
second_model = XGBRegressor(n_estimators=1000, random_state=1, n_jobs = 16).fit(new_train_X, new_train_y)

In [4]:
test = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')
test['abs_lon_change'] = abs(test.dropoff_longitude - test.pickup_longitude)
test['abs_lat_change'] = abs(test.dropoff_latitude - test.pickup_latitude)
X_test = test[features_2]

preds_test = second_model.predict(X_test)
submission_df=pd.DataFrame(test['key'])
submission_df['fare_amount']=pd.Series(preds_test)
submission_df.to_csv('submission.csv',index=False)